# Project Notebook
Generated from script.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to sys.path
# Assuming notebook is in 'notebooks/' and project root is one level up
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
from pokemon_predictor import config
from pokemon_predictor.data_pipeline.tabular import load_data
import pandas as pd
import numpy as np
import joblib


# Qualitative Scenario Testing

In [ ]:
from pokemon_predictor.modeling.predict import PokemonPredictor
from pokemon_predictor import config
import pandas as pd

predictor = PokemonPredictor()

# Scenarios
scenarios = {
    "Mono-Type": ["Charmander", "Squirtle"],
    "Dual-Type": ["Charizard", "Gengar"],
    "Impostors": ["Sudowoodo", "Groudon"]
}

print(f"{'Pokemon':<15} | {'True Type':<20} | {'XGBoost':<20} | {'MLP':<20}")
print("-" * 80)

df_meta = pd.read_csv(config.PROCESSED_DATA_DIR / "pokemon_metadata.csv")

for cat, names in scenarios.items():
    print(f"--- {cat} ---")
    for name in names:
        row = df_meta[df_meta['name'].str.lower() == name.lower()]
        if row.empty: continue
        
        img_path = config.RAW_DATA_DIR / f"{row.iloc[0]['name']}.png"
        pred = predictor.predict(str(img_path))
        
        # True Types
        t1 = row.iloc[0]['type1']
        t2 = row.iloc[0]['type2']
        true_t = f"{t1}" + (f", {t2}" if pd.notna(t2) else "")
        
        if pred:
            xgb = ", ".join(pred['xgboost'])
            mlp = ", ".join(pred['mlp'])
            print(f"{name:<15} | {true_t:<20} | {xgb:<20} | {mlp:<20}")
